In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import transbigdata as tbd
import os
import glob 
from shapely.wkt import loads
import warnings
from shapely import Point
warnings.filterwarnings('ignore')

In [48]:
# 读取等时圈数据
contour = gpd.read_file('./data/contour_15.json')
# 读取中促盟数据
charge_station = gpd.read_file('./data/上海充电桩_中促盟_处理后.geojson')
charge_station.rename(columns={'经度':'lon','纬度':'lat'},inplace=True)
charge_station['station_sum'] = charge_station['交流桩数量'] + charge_station['直流桩数量']
## 删除没用的充电站类型数据
charge_station = charge_station[charge_station['充电站类型']!='04环卫物流车专用']
## lon, lat统一使用wgs 84 坐标系
charge_station.crs = {'init':'epsg:4326'}

In [49]:
contour.rename(columns={'geometry':'geometry_contour'},inplace=True)
contour['geometry'] = contour.apply(lambda row: Point(row['HBLON'], row['HBLAT']), axis=1)
# 找到对应最近的等时圈范围
charge_station_contour = tbd.ckdnearest_point(charge_station,contour)
charge_station_contour.drop(columns=['index','contour', 'LONCOL',
       'LATCOL', 'HBLON', 'HBLAT','geometry_y'],inplace=True)
charge_station_contour.drop(columns=['geometry_x','dist'],inplace=True)
charge_station_contour.rename(columns={'geometry_contour':'geometry'},inplace=True)

In [53]:
charge_station_contour

,myid,充电站名称,充电站位置,充电站投入使用时间,充电站所处道路属性,lon,lat,充电站所属区域分类,充电站类型,交流桩数量,直流桩数量,交流桩总装机功率,直流桩总装机功率,station_sum,geometry
0,0,上海金山石化十五村充电项目,上海市市辖区金山区石化街道金山城市沙滩石化十五村,2023-09-27,01城市内道路,121.347914,30.712635,01充电场站,05其他专用,6.0,0.0,42.0,0,6.0,"POLYGON ((121.35541 30.76452, 121.35786 30.761..."
1,1,上海金山石化十一村小区充电站,上海市市辖区金山区石化街道金山城市沙滩石化十一村,2023-09-27,01城市内道路,121.348137,30.715199,01充电场站,05其他专用,6.0,0.0,42.0,0,6.0,"POLYGON ((121.35541 30.76452, 121.35786 30.761..."
2,2,石化隆安市场停车场,石化街道沪杭公路7958号,2021-09-16,01城市内道路,121.344560,30.716703,10大型建筑物配建停车场,01公共,12.0,0.0,42.0,0,12.0,"POLYGON ((121.31593 30.78854, 121.31796 30.783..."
3,9,上海金山石化二村小区充电站,上海市市辖区金山区石化街道石化二村,2023-09-27,01城市内道路,121.344358,30.719738,01充电场站,05其他专用,6.0,0.0,42.0,0,6.0,"POLYGON ((121.31593 30.78854, 121.31796 30.783..."
4,11,上海金山石化三村小区充电站,上海市市辖区金山区石化街道石化三村小区,2023-09-27,01城市内道路,121.343585,30.721829,01充电场站,05其他专用,4.0,0.0,28.0,0,4.0,"POLYGON ((121.31593 30.78854, 121.31796 30.783..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19585,19584,崇明东平度假村充电站,东风公路555号,2023-02-27,None,121.493241,31.706818,03公共机构内部停车场,01公共,8.0,0.0,56.0,0,8.0,"POLYGON ((121.40946 31.76973, 121.41418 31.766..."
19586,19585,崇明庙镇充电站,庙镇人民政府x533南,2023-02-27,None,121.349042,31.712411,03公共机构内部停车场,01公共,12.0,0.0,84.0,0,12.0,"POLYGON ((121.33386 31.78150, 121.33613 31.781..."
19587,19586,新海镇快充站,崇明区新海镇北沿公路3366号,2021-08-29,None,121.293068,31.818175,03公共机构内部停车场,01公共,1.0,2.0,7.0,120,3.0,"POLYGON ((121.33027 31.84820, 121.33240 31.848..."
19588,19589,城中村改造二期A6d-06a地块,京华路徐孝路,2023-07-01,None,121.003809,30.963185,07交通枢纽公共停车场,01公共,4.0,3.0,28.0,180,7.0,"POLYGON ((121.04767 31.00329, 121.05040 31.002..."


#### 识别附近充电站数量

In [59]:
station_location = charge_station_contour[['lon','lat','myid','交流桩数量','直流桩数量']]
station_geometry = charge_station_contour[['geometry','myid']]
station_geometry = gpd.GeoDataFrame(station_geometry,geometry='geometry')
station_location['geometry'] = station_location.apply(lambda row: Point(row['lon'],row['lat']), axis=1)
station_location = gpd.GeoDataFrame(station_location,geometry='geometry')

In [62]:
from tqdm import  tqdm

total_rows = len(station_geometry)
# 定义每批次处理的行数
batch_size = 100
num_batches = (total_rows + batch_size - 1) // batch_size
station_num = pd.DataFrame()
# 使用 tqdm 来显示进度条
for i in tqdm(range(num_batches), desc='Processing batches'):
    # 计算当前批次的起始和结束索引
    start_index = i * batch_size
    end_index = min((i + 1) * batch_size, total_rows)
    
    # 取出当前批次的数据
    batch_data = station_geometry.iloc[start_index:end_index].copy()
    station_count = gpd.sjoin(batch_data,station_location)
    station_count.rename(columns={'myid_left':'myid'},inplace=True)
    station_count = station_count.groupby('myid').agg({
    'myid_right':'count',  # 对'myid_right'列进行计数
    '交流桩数量':'sum',   # 对直流桩数量列进行求和
    '直流桩数量':'sum'}
).reset_index()
    station_num = pd.concat([station_num,station_count])
station_num

Processing batches: 100%|██████████| 196/196 [00:11<00:00, 17.41it/s]


,myid,myid_right,交流桩数量,直流桩数量
0,0,143,842.0,379.0
1,1,143,842.0,379.0
2,2,198,1329.0,420.0
3,3,173,1203.0,266.0
4,4,173,1203.0,266.0
...,...,...,...,...
85,19585,32,249.0,15.0
86,19586,14,37.0,8.0
87,19587,144,1060.0,351.0
88,19589,7,11.0,3.0


In [63]:
station_num.rename(columns={'myid_right':'Number_of_Charging_Stations_Nearby'},inplace=True)

In [37]:
test = station_geometry.iloc[:2].copy()
test = gpd.sjoin(test,station_location)
test.rename(columns={'myid_left':'myid'},inplace=True)
test = test.groupby('myid')['myid_right'].count().reset_index()
test

,myid,myid_right
0,0,143
1,1,143


In [4]:
shanghai_area = pd.read_csv('./data/shanghai.csv')
## 转换成polygon数据
shanghai_area['geometry'] = shanghai_area['geometry'].apply(lambda x: loads(x) if isinstance(x, object) else None)
## dataframe转换成geodataframe
shanghai_area = gpd.GeoDataFrame(shanghai_area,geometry='geometry')
shanghai_area.drop(columns=['index_right'],inplace=True)
## 计算建筑物面积
def building_area(gdf):
    gdf.crs = {'init':'epsg:4326'}
    ## 使用适配于上海的投影坐标系
    tmp = gdf.to_crs(epsg=4528)
    gdf['area'] = tmp.area
    ## 计算总面积
    gdf['area'] = gdf['area'] * gdf['height']
    return gdf
shanghai_area = building_area(shanghai_area)
charge_station_contour = gpd.GeoDataFrame(charge_station_contour,geometry='geometry')

#### 数据量太大了，拆分成多个小数据来算

In [5]:
# from tqdm import  tqdm

# total_rows = len(charge_station_contour)
# # 定义每批次处理的行数
# batch_size = 100
# num_batches = (total_rows + batch_size - 1) // batch_size
# data = pd.DataFrame()
# # 使用 tqdm 来显示进度条
# for i in tqdm(range(num_batches), desc='Processing batches'):
#     # 计算当前批次的起始和结束索引
#     start_index = i * batch_size
#     end_index = min((i + 1) * batch_size, total_rows)
    
#     # 取出当前批次的数据
#     batch_data = charge_station_contour.iloc[start_index:end_index].copy()
#     station_sjoined_area = gpd.sjoin(batch_data,shanghai_area)
#     station_sjoined_area.drop(columns=['index_right','building_id'],inplace=True)
#     station_sjoined_area = station_sjoined_area.groupby(['myid'])['area'].sum().reset_index()
#     data = pd.concat([data,station_sjoined_area])
# data
# data.to_csv('./data/tmp_area.csv',index=None)

Processing batches: 100%|██████████| 196/196 [2:34:48<00:00, 47.39s/it]    


,myid,area
0,0,5.975102e+07
1,1,5.975102e+07
2,2,9.012970e+07
3,3,9.157995e+07
4,4,9.157995e+07
...,...,...
85,19585,9.122114e+06
86,19586,5.508799e+06
87,19587,6.896917e+07
88,19589,8.189128e+06


In [6]:
# from concurrent.futures import ThreadPoolExecutor

# # 假设 charge_station_contour 和 shanghai_area 已经被定义

# def process_batch(start_index, end_index):
#     # 取出当前批次的数据
#     batch_data = charge_station_contour.iloc[start_index:end_index].copy()
#     station_sjoined_area = gpd.sjoin(batch_data, shanghai_area.copy())
#     station_sjoined_area.drop(columns=['index_right', 'building_id'], inplace=True)
#     station_sjoined_area = station_sjoined_area.groupby(['myid'])['area'].sum().reset_index()
#     return station_sjoined_area
# total_rows = len(charge_station_contour)
# # 定义每批次处理的行数
# batch_size = 100
# # 计算批次数量
# num_batches = (total_rows + batch_size - 1) // batch_size
# # 创建一个空的 DataFrame 来存储结果
# data = pd.DataFrame()
# # 使用 ThreadPoolExecutor 来执行多线程处理
# with ThreadPoolExecutor(max_workers=24) as executor:  # 你可以根据你的CPU核心数来设置 max_workers
#     # 使用列表推导式和 submit 方法来安排任务
#     futures = [executor.submit(process_batch, i * batch_size, min((i + 1) * batch_size, total_rows))
#                for i in range(num_batches)]
#     completed = 0  # 完成的任务数量
#     for future in futures:
#         batch_result = future.result()
#         data = pd.concat([data, batch_result])
#         completed +=1
#         print(f"Completed {completed} of {num_batches} batches ({completed * 100 / num_batches:.2f}%)")

Completed 1 of 196 batches (0.51%)
Completed 2 of 196 batches (1.02%)
Completed 3 of 196 batches (1.53%)
Completed 4 of 196 batches (2.04%)
Completed 5 of 196 batches (2.55%)
Completed 6 of 196 batches (3.06%)
Completed 7 of 196 batches (3.57%)
Completed 8 of 196 batches (4.08%)
Completed 9 of 196 batches (4.59%)
Completed 10 of 196 batches (5.10%)
Completed 11 of 196 batches (5.61%)
Completed 12 of 196 batches (6.12%)
Completed 13 of 196 batches (6.63%)
Completed 14 of 196 batches (7.14%)
Completed 15 of 196 batches (7.65%)
Completed 16 of 196 batches (8.16%)


#### 读取上海市建筑数据信息

In [6]:
# shanghai_area = pd.read_csv('./data/shanghai.csv')
# ## 转换成polygon数据
# shanghai_area['geometry'] = shanghai_area['geometry'].apply(lambda x: loads(x) if isinstance(x, object) else None)
# ## dataframe转换成geodataframe
# shanghai_area = gpd.GeoDataFrame(shanghai_area,geometry='geometry')
# shanghai_area.drop(columns=['index_right'],inplace=True)
# ## 计算建筑物面积
# def building_area(gdf):
#     gdf.crs = {'init':'epsg:4326'}
#     ## 使用适配于上海的投影坐标系
#     tmp = gdf.to_crs(epsg=4528)
#     gdf['area'] = tmp.area
#     ## 计算总面积
#     gdf['area'] = gdf['area'] * gdf['height']
#     return gdf
# shanghai_area = building_area(shanghai_area)
# charge_station_contour = gpd.GeoDataFrame(charge_station_contour,geometry='geometry')
# station_sjoined_area = gpd.sjoin(charge_station_contour,shanghai_area)
# station_sjoined_area.drop(columns=['index_right','building_id'],inplace=True)
# station_sjoined_area = station_sjoined_area.groupby(['myid'])['area'].sum().reset_index()
# station_sjoined_area

#### 读取POI数据信息

In [5]:
def read_poi(folder_path):
    csv_file_pattern = os.path.join(folder_path, '*.csv')  # 构建文件匹配模式
    csv_files = glob.glob(csv_file_pattern)  # 获取所有.csv文件的路径列表
    data = pd.DataFrame()
    for file_path in csv_files:
        df = pd.read_csv(file_path,on_bad_lines='skip')
        ## 处理tsv文件
        if len(df.columns) == 1:
            df = pd.read_csv(file_path,sep='\t')
        data = pd.concat([data,df])
    return data

In [6]:
poi1 = read_poi('./POI')
poi2 = read_poi('./POI2')
poi3 = read_poi('./POI长三角')
poi1 = poi1[poi1['province']=='上海市']
poi2 = poi2[poi2['province']=='上海市']
poi3 = poi3[poi3['province']=='上海市']
## 合并上海市所有的POI信息
poi_shanghai = pd.concat([poi1,poi2,poi3])
poi_shanghai.drop_duplicates(subset=['uid'],inplace=True)
poi_shanghai = gpd.GeoDataFrame(poi_shanghai,
                                geometry=gpd.points_from_xy(poi_shanghai['lon'],poi_shanghai['lat']))

In [7]:
tech_park_poi = poi_shanghai[poi_shanghai['name'].str.contains('科技园')]
tech_park_poi['technology_park'] = 1
tech_park_poi = tech_park_poi[['technology_park','geometry']]

car_park_poi = poi_shanghai[poi_shanghai['name'].str.contains('停车场')]
car_park_poi['car_park'] = 1
car_park_poi = car_park_poi[['car_park','geometry']]

supermarket_poi = poi_shanghai[(poi_shanghai['name'].str.contains('超市'))&(poi_shanghai['tag'].str.contains('超市'))]
supermarket_poi['supermarket'] = 1
supermarket_poi = supermarket_poi[['supermarket','geometry']]

tag_not_nan_poi = poi_shanghai.dropna(subset=['tag'])
residential_poi = tag_not_nan_poi[tag_not_nan_poi['tag'].str.contains('住宅区')]
residential_poi['residential'] = 1
residential_poi = residential_poi[['residential','geometry']]

commercial_poi = poi_shanghai[poi_shanghai['type']=='shopping']
commercial_poi['commercial'] = 1
commercial_poi = commercial_poi[['commercial','geometry']]

In [8]:
from tqdm import tqdm 
def merge_info_to_charge_station(charge_station,merge_poi,column):
    total_rows = len(charge_station)
    # 定义每批次处理的行数
    batch_size = 100
    num_batches = (total_rows + batch_size - 1) // batch_size
    data = pd.DataFrame()
    # 使用 tqdm 来显示进度条
    for i in tqdm(range(num_batches), desc='Processing batches'):
        # 计算当前批次的起始和结束索引
        start_index = i * batch_size
        end_index = min((i + 1) * batch_size, total_rows)
        
        # 取出当前批次的数据
        batch_data = charge_station.iloc[start_index:end_index].copy()
        tmp_sjoin = gpd.sjoin(batch_data,merge_poi,how='left',op='intersects')
        tmp_sjoin[column] = tmp_sjoin[column].fillna(0)
        data = pd.concat([data,tmp_sjoin])
    myid_agg = data.groupby(['myid'])[column].sum().reset_index()
    return myid_agg

In [9]:
technology_agg = merge_info_to_charge_station(charge_station_contour,tech_park_poi,'technology_park')

Processing batches: 100%|██████████| 196/196 [00:06<00:00, 29.64it/s]


In [11]:
car_agg = merge_info_to_charge_station(charge_station_contour,car_park_poi,'car_park')

Processing batches: 100%|██████████| 196/196 [03:00<00:00,  1.08it/s]


In [12]:
supermarket_agg = merge_info_to_charge_station(charge_station_contour,supermarket_poi,'supermarket')

Processing batches: 100%|██████████| 196/196 [00:34<00:00,  5.68it/s]


In [13]:
residential_agg = merge_info_to_charge_station(charge_station_contour,residential_poi,'residential')


Processing batches: 100%|██████████| 196/196 [02:11<00:00,  1.49it/s]


In [14]:
commercial_agg = merge_info_to_charge_station(charge_station_contour,commercial_poi,'commercial')

Processing batches: 100%|██████████| 196/196 [2:07:51<00:00, 39.14s/it]     


In [15]:
geographic_info = pd.merge(technology_agg,car_agg,on=['myid'],how='inner')
geographic_info = pd.merge(geographic_info,supermarket_agg,on=['myid'],how='inner')
geographic_info = pd.merge(geographic_info,residential_agg,on=['myid'],how='inner')
geographic_info = pd.merge(geographic_info,commercial_agg,on=['myid'],how='inner')
## 合并地理信息
charge_station_feature = pd.merge(charge_station_contour,geographic_info,on=['myid'],how='inner')
charge_station_feature.drop(columns=['充电站位置', '充电站投入使用时间'],inplace=True)

In [16]:
charge_station_feature.to_csv('./output/中促盟等时圈15充电站特征.csv',index=None)

In [19]:
# # 合并面积信息
# charge_station = pd.read_csv('./output/中促盟等时圈15充电站特征.csv')
# area= pd.read_csv('./data/tmp_area.csv')
# charge_station = pd.merge(charge_station,area,on=['myid'],how='inner')
# charge_station.to_csv('./output/中促盟等时圈15充电站特征.csv',index=None)

In [65]:
charge_station = pd.read_csv('./output/中促盟等时圈15充电站特征.csv')
charge_station.drop(columns=['Number_of_Charging_Stations_Nearby'],inplace=True)
charge_station = pd.merge(charge_station,station_num,on='myid',how='inner')
charge_station.to_csv('./output/中促盟等时圈15充电站特征.csv',index=None)
charge_station

,myid,充电站名称,充电站所处道路属性,lon,lat,充电站所属区域分类,充电站类型,交流桩数量_x,直流桩数量_x,交流桩总装机功率,...,geometry,technology_park,car_park,supermarket,residential,commercial,area,Number_of_Charging_Stations_Nearby,交流桩数量_y,直流桩数量_y
0,0,上海金山石化十五村充电项目,01城市内道路,121.347914,30.712635,01充电场站,05其他专用,6.0,0.0,42.0,...,"POLYGON ((121.355413 30.764519, 121.357863 30....",0.0,128.0,67.0,138.0,2025.0,5.975102e+07,143,842.0,379.0
1,1,上海金山石化十一村小区充电站,01城市内道路,121.348137,30.715199,01充电场站,05其他专用,6.0,0.0,42.0,...,"POLYGON ((121.355413 30.764519, 121.357863 30....",0.0,128.0,67.0,138.0,2025.0,5.975102e+07,143,842.0,379.0
2,2,石化隆安市场停车场,01城市内道路,121.344560,30.716703,10大型建筑物配建停车场,01公共,12.0,0.0,42.0,...,"POLYGON ((121.315933 30.788539, 121.317958 30....",0.0,144.0,84.0,171.0,2595.0,9.012970e+07,198,1329.0,420.0
3,9,上海金山石化二村小区充电站,01城市内道路,121.344358,30.719738,01充电场站,05其他专用,6.0,0.0,42.0,...,"POLYGON ((121.315933 30.788539, 121.317958 30....",0.0,144.0,84.0,171.0,2595.0,9.012970e+07,198,1329.0,420.0
4,11,上海金山石化三村小区充电站,01城市内道路,121.343585,30.721829,01充电场站,05其他专用,4.0,0.0,28.0,...,"POLYGON ((121.315933 30.788539, 121.317958 30....",0.0,144.0,84.0,171.0,2595.0,9.012970e+07,198,1329.0,420.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19583,19584,崇明东平度假村充电站,NaN,121.493241,31.706818,03公共机构内部停车场,01公共,8.0,0.0,56.0,...,"POLYGON ((121.409464 31.769731, 121.414176 31....",1.0,19.0,8.0,23.0,53.0,7.830585e+06,50,328.0,128.0
19584,19585,崇明庙镇充电站,NaN,121.349042,31.712411,03公共机构内部停车场,01公共,12.0,0.0,84.0,...,"POLYGON ((121.333863 31.781504, 121.336131 31....",0.0,5.0,10.0,8.0,181.0,9.122114e+06,32,249.0,15.0
19585,19586,新海镇快充站,NaN,121.293068,31.818175,03公共机构内部停车场,01公共,1.0,2.0,7.0,...,"POLYGON ((121.330267 31.8482, 121.332399 31.84...",0.0,9.0,5.0,15.0,54.0,5.508799e+06,14,37.0,8.0
19586,19589,城中村改造二期A6d-06a地块,NaN,121.003809,30.963185,07交通枢纽公共停车场,01公共,4.0,3.0,28.0,...,"POLYGON ((121.047669 31.003286, 121.050402 31....",0.0,9.0,10.0,7.0,106.0,8.189128e+06,7,11.0,3.0
